Threshold는 문턱이라는 뜻으로 어떤 기준에 해당하는 문턱값(Thresholding Value)를 기준으로 상황이 급격하게 변화되는 것을 나타낸다. 예를 들어 반올림을 생각하면 0.5라는 숫자를 기준으로 이보다 작으면 0, 크면 1이 되는 것을 봤을 때 0.5가 문턱값이 된다. 

이미지 프로세싱에 thresholding을 적용하기 위해 grayscale 이미지로 변환하여 적용한다.

# Global Thresholding

opencv에서 하나의 이미지에 전역으로 적용될 하나의 문턱값을 이용해 thresholding 기능을 제공하는 함수가 있다.

cv2.threshold(img, thoeshold_value,value,flag)
- img : Grayscale 이미지
- threshold_value : 픽셀문턱값
- value : 픽셀 문턱값보다 클 때 적용되는 최대값(flag에 따라 적용된다.)
- flag : 문턱값 적용 방법 
   
    1) cv2.THRESH_BINARY:픽셀값이 threshold_value 보다 크면 value, 작으면 0 할당

    2) cv2.THRESH_BINARY_INV: 픽셀값이 threshold_value보다 크면 0, 작으면  value 할당

    3) cv2.THRESH_TRUNC : 픽셀값이 threshold_value 보다 크면 threshold_value, 작으면 픽셀 값 그대로 할당

    4) cv2.THRESH_TOERO : 픽셀값이 threshold_value 보다 크면 픽셀값 그대로, 작으면 0 할당
   
    5) cv2.THRESH_TOZERO_INV : 픽셀값이 threshold_value보다 크면 0, 작으면 픽셀 값 그대로 할당

In [4]:
import numpy as np
import cv2

img = cv2.imread('image/Lenna.png', cv2.IMREAD_GRAYSCALE)

ret, thr1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
ret, thr2 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
ret, thr3 = cv2.threshold(img, 127, 255, cv2.THRESH_TRUNC)
ret, thr4 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)
ret, thr5 = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO_INV)

cv2.imshow('original',img)
cv2.imshow('BINARY', thr1)
cv2.imshow('BINARY_INV',thr2)
cv2.imshow('TRUNC',thr3)
cv2.imshow('TOZERO',thr4)
cv2.imshow('TOZERO_INV',thr5)

cv2.waitKey(0)
cv2.destroyAllWindows()


# Adaptive Thresholding

앞에 설명한 cv2.threshold()함수를 이용한 global thresholding방법은 이미지 전반에 걸쳐 적용되는 하나의 문턱값을 활용한 로직을 사용했습니다. 이 방법은 이미지의 각 부분의 광원 조건에 따라 이미지 프로세싱을 함에 있어 효과적인 방법이 아닐 수 있습니다. 

Adaptive Thresholding은 이미지의 서로 다른 작은 역역에  적용되는 문턱값을 계산하고 이를 이미지에 적용함으로써 보다 나은 결과를 도출하는데 사용되는 방법입니다. 


cv2.adaptiveThreshold(img, value, adaptivMethod, thresholdType,blocksize,C)

- img : Grayscale 이미지
- value : adaptiveMethod에 의해 계산된 문턱값과 thresholdType에 의해 픽셀에 적용될 최대값
- adaptiveMethod : 사용할 Adaptive Thresholding 알고리즘

    1) cv2.ADAPTIVE_THRESH_MEAN_C:적용할 픽셀(x,y)를 중심으로 하는 blocksize * blocksize 안에 있는 픽셀값의 평균에서 C를 뺀 값을 문턱값으로 함
    
    2) cv2.ADAPTIVE_THRESH_GAUSSIAN_C: 적용할 픽셀(x,y)를 중심으로 하는 blocksize * blocksize 안에 있는 gaussian 윈도우 기반 가중치들의 합에서 c를 뺀 값을 문턱값으로 함


- blocksize : 픽셀에 적용할 문턱값을 계산하기 위한 블럭 크기, 적용될 픽셀이 블럭의 중심이 됨, 따라서 blocksize는 홀수여야 함
-  C : 보정상수로 이 값이 양수이면 adaptive 문턱값에서 빼고, 음수이면 더해줌

In [27]:
import numpy as np
import cv2

def thresholding():
    img = cv2.imread('image/Lenna.png', cv2.IMREAD_GRAYSCALE)
    
    ret, thr1 = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
    thr2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 6)
    thr3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 6)
    
    titles = [ 'original', 'Global Thresholding(v=127)','Adaptive  MEAN', 'Adaptive GAUSSIAN']
    images = [img, thr1, thr2, thr3]
    
    for i in range(4):
        cv2.namedWindow(titles[i],cv2.WINDOW_NORMAL)
        cv2.imshow(titles[i], images[i])
        
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
thresholding()

# Otus's Binariztion

global thresholding 방법에서 문턱값으로 우리가 정한 임이의 값을 사용했습니다. 그런데 어떤 이미지에 적용되어 가장 좋은 결과를 내놓게 될 문턱값은 어떻게 얻을 수 있을까요? 정답은 시행착오를 거치는 방법입니다. 

만약 이미지 히스토그램이 두개의 봉우리를 가지는 bimodal이미지라고 하면 이 이미지에 대한 문턱값으로 두 봉우리 사이의 값을 취하면 가장 좋은 결과를 얻을 수 있습니다. 

Otsu Binarization은 이미지 히스토그램을 분석한 후 중간값을 취하여 thresholding 합니다.


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def thresholding():
    img =  cv2.imread('image/Lenna.png',cv2.IMREAD_GRAYSCALE)
    
    #전역 thresholding 적용
    ret, thr1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    
    # Otsu 바이너리제이션 (value를 0, flag에 cv2.THRSH_OTSU더함)
    ret, thr2 = cv2.threshold(img, 0, 255, cv2.THRSH_BINARY+cv2.THRESH_OTSU)
    
    #가우시안 불러 적용 후 Otus 바이너리제이션
    blur = cv2.GaussianBlur(img, (5,5), 0)
    ret, thr3 = cv2.threshold(blur, 0, 255,, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    titles = ['original noisy', ]